In [1]:
# ignore this cell, this is just a helper cell to provide the magic %highlight_file
%run ../highlighter.py

# Validate

This playbook uses [napalm validation](http://napalm.readthedocs.io/en/latest/validate/index.html) functionality to verify correctness of the network.

## Code

In [7]:
%highlight_file validate.py

## Demo

Let's start by running the script on an unconfigured network:

In [3]:
%run validate.py

* spine00.cmh ** changed : False ***********************************************
---- validate ** changed : False  ----------------------------------------------
---- read data ** changed : False  ---------------------------------------------
{ 'interfaces': { 'Ethernet1': { 'connects_to': 'leaf00.cmh',
                                 'enabled': True,
                                 'ipv4': '10.0.0.0/31'},
                  'Ethernet2': { 'connects_to': 'leaf01.cmh',
                                 'enabled': True,
                                 'ipv4': '10.0.0.2/31'}},
  'sessions': [ {'ipv4': '10.0.0.1', 'peer_as': 65100},
                {'ipv4': '10.0.0.3', 'peer_as': 65101}]}
---- validating data ** changed : False  ---------------------------------------
{ 'complies': False,
  'get_bgp_neighbors': { 'complies': False,
                         'extra': [],
                         'missing': ['global'],
                         'present': {}},
  'skipped': []}

* spine01.cmh 

For each host we get the data we are using for validation and the result. What the report is saying is that we don't even have the BGP instance 'global' (default instance) configured so let's do it:

In [4]:
%run configure.py

**** Playbook to configure the network *****************************************
* spine00.cmh ** changed : True ************************************************
---- Loading Configuration on the device ** changed : True  --------------------
@@ -8,7 +8,8 @@
 !
 transceiver qsfp default-mode 4x10G
 !
-hostname blah
+hostname spine00.cmh
+ip domain-name cmh.acme.com
 !
 spanning-tree mode mstp
 !
@@ -20,13 +21,28 @@
 username vagrant privilege 15 role network-admin secret sha512 $6$kRQZJTqx69hOW5ag$Y6VX8Kk37TWEsriKdr6ixqvMuUSSbuFu2Eh/5SIet2TCeXP3bdlwikIAruPp6lHB5HdC.t6tPsZVctHMU7H590
 !
 interface Ethernet1
+   description link to leaf00.cmh
+   no switchport
+   ip address 10.0.0.0/31
 !
 interface Ethernet2
+   description link to leaf01.cmh
+   no switchport
+   ip address 10.0.0.2/31
 !
 interface Management1
    ip address 10.0.2.15/24
 !
-no ip routing
+ip routing
+!
+router bgp 65000
+   neighbor 10.0.0.1 remote-as 65100
+   neighbor 10.0.0.1 maximum-routes 12000 
+   neighbor 10

Now the network is configured let's validate the deployment again:

In [6]:
%run validate.py

* spine00.cmh ** changed : False ***********************************************
---- validate ** changed : False  ----------------------------------------------
---- read data ** changed : False  ---------------------------------------------
{ 'interfaces': { 'Ethernet1': { 'connects_to': 'leaf00.cmh',
                                 'enabled': True,
                                 'ipv4': '10.0.0.0/31'},
                  'Ethernet2': { 'connects_to': 'leaf01.cmh',
                                 'enabled': True,
                                 'ipv4': '10.0.0.2/31'}},
  'sessions': [ {'ipv4': '10.0.0.1', 'peer_as': 65100},
                {'ipv4': '10.0.0.3', 'peer_as': 65101}]}
---- validating data ** changed : False  ---------------------------------------
{ 'complies': False,
  'get_bgp_neighbors': { 'complies': False,
                         'extra': [],
                         'missing': [],
                         'present': { 'global': { 'complies': False,
            

                                                                                              'missing': [ ],
                                                                                              'present': { '10.0.0.0': { 'complies': False,
                                                                                                                         'diff': { 'complies': False,
                                                                                                                                   'extra': [ ],
                                                                                                                                   'missing': [ ],
                                                                                                                                   'present': { 'is_up': { 'actual_value': False,
                                                                                                                                  

What the report is basically telling us is that ``spina01`` and ``leaf01`` are pssing our tests, however, ``spine00`` and ``leaf00`` as one of their BGP sessions that should be ``up`` is actually ``down``.